### Grid Optimizer: A Reinforcement Learning Approach

In [10]:
import numpy as np
import random
import pandas as pd

In [11]:
# Step 1: Define the Grid Environment with obstacles
class GridEnvironment:
    def __init__(self, size=100, obstacle_prob=0.2):
        self.size = size
        self.grid = np.zeros((size, size))
        self.start = (random.randint(0, size-1), random.randint(0, size-1))
        self.goal = (random.randint(0, size-1), random.randint(0, size-1))
        
        # Set obstacles
        for i in range(size):
            for j in range(size):
                if random.random() < obstacle_prob and (i, j) != self.start and (i, j) != self.goal:
                    self.grid[i, j] = -1  # Represent obstacles by -1

    def is_obstacle(self, state):
        return self.grid[state] == -1

    def is_terminal(self, state):
        return state == self.goal

In [12]:
# Step 2: Define MDP Transition and Reward Functions
class MDP:
    def __init__(self, env):
        self.env = env
        self.actions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Right, Down, Left, Up
    
    def transition(self, state, action):
        next_state = (state[0] + action[0], state[1] + action[1])
        if 0 <= next_state[0] < self.env.size and 0 <= next_state[1] < self.env.size:
            if not self.env.is_obstacle(next_state):
                return next_state
        return state  # Stay in place if hitting an obstacle or boundary

    def reward(self, state, action, next_state):
        return 1 if next_state == self.env.goal else -0.1  # Small penalty for each step


In [13]:
# Step 3: Implement Policy Iteration Agent
class PolicyIterationAgent:
    def __init__(self, mdp, gamma=0.9, theta=1e-3):
        self.mdp = mdp
        self.gamma = gamma
        self.theta = theta
        self.value_table = np.zeros((self.mdp.env.size, self.mdp.env.size))
        self.policy = np.random.choice(range(len(self.mdp.actions)), (self.mdp.env.size, self.mdp.env.size))
    
    def policy_evaluation(self):
        while True:
            delta = 0
            for i in range(self.mdp.env.size):
                for j in range(self.mdp.env.size):
                    state = (i, j)
                    if self.mdp.env.is_terminal(state) or self.mdp.env.is_obstacle(state):
                        continue
                    
                    action = self.mdp.actions[self.policy[i, j]]
                    next_state = self.mdp.transition(state, action)
                    reward = self.mdp.reward(state, action, next_state)
                    
                    v = self.value_table[i, j]
                    self.value_table[i, j] = reward + self.gamma * self.value_table[next_state]
                    delta = max(delta, abs(v - self.value_table[i, j]))
            if delta < self.theta:
                break

    def policy_improvement(self):
        policy_stable = True
        for i in range(self.mdp.env.size):
            for j in range(self.mdp.env.size):
                state = (i, j)
                if self.mdp.env.is_terminal(state) or self.mdp.env.is_obstacle(state):
                    continue
                
                old_action = self.policy[i, j]
                action_values = []
                for action in self.mdp.actions:
                    next_state = self.mdp.transition(state, action)
                    reward = self.mdp.reward(state, action, next_state)
                    action_values.append(reward + self.gamma * self.value_table[next_state])
                
                self.policy[i, j] = np.argmax(action_values)
                if old_action != self.policy[i, j]:
                    policy_stable = False
        return policy_stable

    def policy_iteration(self):
        while True:
            self.policy_evaluation()
            if self.policy_improvement():
                break

In [14]:
# Step 4: Implement Q-Learning Agent
class QLearningAgent:
    def __init__(self, mdp, alpha=0.1, gamma=0.9, epsilon=0.1, episodes=1000):
        self.mdp = mdp
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.episodes = episodes
        self.q_table = np.zeros((self.mdp.env.size, self.mdp.env.size, len(self.mdp.actions)))
    
    def train(self):
        for episode in range(self.episodes):
            state = self.mdp.env.start
            while not self.mdp.env.is_terminal(state):
                if random.uniform(0, 1) < self.epsilon:
                    action_idx = random.randint(0, len(self.mdp.actions) - 1)
                else:
                    action_idx = np.argmax(self.q_table[state[0], state[1]])
                
                action = self.mdp.actions[action_idx]
                next_state = self.mdp.transition(state, action)
                reward = self.mdp.reward(state, action, next_state)
                
                best_next_action = np.argmax(self.q_table[next_state[0], next_state[1]])
                td_target = reward + self.gamma * self.q_table[next_state[0], next_state[1], best_next_action]
                td_delta = td_target - self.q_table[state[0], state[1], action_idx]
                self.q_table[state[0], state[1], action_idx] += self.alpha * td_delta
                
                state = next_state

In [16]:
# Initialize environment and MDP
env = GridEnvironment()
mdp = MDP(env)

# Run Policy Iteration
pi_agent = PolicyIterationAgent(mdp)
pi_agent.policy_iteration()

# Run Q-Learning
ql_agent = QLearningAgent(mdp)
ql_agent.train()

# Compare results by examining value tables or testing optimal policies
print("Policy Iteration Value Table:", pi_agent.value_table)
print("Q-Learning Q-Table:", ql_agent.q_table)


Policy Iteration Value Table: [[-0.99997942 -0.99997713 -0.99997459 ... -0.99999936 -0.99999943
  -0.99999948]
 [ 0.         -0.99997459 -0.99997177 ... -0.99999929 -0.99999936
  -0.99999943]
 [-0.99997459 -0.99997177 -0.99996863 ... -0.99999921 -0.99999929
  -0.99999936]
 ...
 [-0.98969245 -0.98854717 -0.98727463 ... -0.99979101 -0.99981191
  -0.99983072]
 [-0.9907232  -0.98969245 -0.98854717 ... -0.99981191 -0.99983072
  -0.99984765]
 [-0.99165088 -0.9907232  -0.98969245 ... -0.99983072 -0.99984765
  -0.99986288]]
Q-Learning Q-Table: [[[-0.44653411 -0.44592607 -0.44479777 -0.4414602 ]
  [-0.44640663 -0.44816793 -0.44332749 -0.44618465]
  [-0.44694058 -0.44601725 -0.44923204 -0.44578683]
  ...
  [-0.41917654 -0.4180863  -0.42200271 -0.41775029]
  [-0.41895861 -0.41832715 -0.42159233 -0.41692624]
  [-0.41685579 -0.41912154 -0.42221073 -0.41816637]]

 [[ 0.          0.          0.          0.        ]
  [-0.44614612 -0.44859219 -0.44695701 -0.44444413]
  [-0.44900991 -0.44529491 -0.4455